In [1]:
import gym
import numpy as np
import random
import torch
import time
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
map1 = ['GASBG']


In [4]:
map2 = ['GBSAG']


In [5]:

env1 = gym.make('FrozenLake-v0', is_slippery=False, desc = map1)

env1.render()


GASBG


In [6]:

env2 = gym.make('FrozenLake-v0', is_slippery=False, desc = map2)

env2.render()


GBSAG


In [7]:
# Total number of States and Actions
n_states = env1.observation_space.n
n_actions = 4
n_rows = 1
n_cols = 5
print( "States = ", n_states)
print( "Actions = ", n_actions)

States =  5
Actions =  4


In [8]:
# Total number of States and Actions
n_states = env2.observation_space.n
n_actions = 4
n_rows = 1
n_cols = 5
print( "States = ", n_states)
print( "Actions = ", n_actions)

States =  5
Actions =  4


In [9]:
def restrict_actions(Q, n_states, n_rows):

  Q.at[n_states -1, :] = np.zeros(n_actions,)
  Q.at[0, :] = np.zeros(n_actions,)
  for i in range( 0, n_states, n_rows): 
    Q.at[i,0] = np.NaN
  for i in range( n_rows -1 , n_states, n_rows): 
    Q.at[i,2] = np.NaN
  for i in range(0, n_rows):
    Q.at[i,3] = np.NaN
  for i in range(n_states - n_rows , n_states):
    Q.at[i,1 ]= np.NaN
  
  return Q


In [10]:
def choose_action(Q, state, epsilon):
  random_for_epsilon = np.random.rand()
  if random_for_epsilon <= epsilon:
    action = random.choice((0,2))
  elif Q.loc[state,2] > Q.loc[state,0]: 
   action = 2
  elif Q.loc[state,0] > Q.loc[state,2]: 
    action = 0
  elif Q.loc[state,0] == Q.loc[state,2]: 
    action = random.choice((0,2))
  return action

In [11]:
##assign index to each state using state-matrix

state_matrix = np.arange(0,n_states).reshape(n_rows,n_cols)
state_matrix

def rowsandcols(state):
  ''' input: state returned by env
      output: location of state as (row,col) tuple'''
  return int(np.where(state_matrix ==state)[0]), int(np.where(state_matrix ==state)[1])

In [12]:
def rewarder1(new_state, reward, entrance_stimulus):  #ENV1

  if entrance_stimulus == 'A':
    if new_state == 0: 
      reward += 100
    else:
      reward -= 10

  if entrance_stimulus == 'B':
    if new_state == 4:
      reward += 100
    else:
      reward -= 10

  return reward

In [13]:
def rewarder2(new_state, reward, entrance_stimulus): #ENV2

  if entrance_stimulus == 'A':
    if new_state == 4:
      reward += 100
    else:
      reward -= 10

  if entrance_stimulus == 'B':
    if new_state == 0:
      reward += 100
    else:
      reward -= 10

  return reward

In [30]:
reps = 1
num_episodes = 1500
steps_total = [] # store number of steps taken in each episode
rewards_total = [] #store reward obtained for each episode
epsilon_total = [] #store epsilon obtained at the end of each episode
terminal_state = []

In [31]:
for i in range(reps):
  
  epsilon = 0.8
  epsilon_final = 0.1
  epsilon_decay = 0.99

  gamma = 0.90 # discount factor
  learning_rate = 0.9 #how important is the difference between q-val from q-table and what's observed

  Q1 = pd.DataFrame(np.random.rand(n_states,n_actions)/1000)
  Q2 = pd.DataFrame(np.random.rand(n_states,n_actions)/1000)
  Q3 = pd.DataFrame(np.random.rand(n_states,n_actions)/1000)
  Q4 = pd.DataFrame(np.random.rand(n_states,n_actions)/1000)
  #Q .loc[n_states-1] = np.zeros(n_actions,)
  #Q.loc[n_cols-1] = np.zeros(n_actions,) 
  #Q = restrict_actions(Q, 5, 1)

  for i_episode in range(num_episodes):

    if np.random.randn() > 0.5: 
      entrance_stimulus = 'A'
    else: 
      entrance_stimulus = 'B'

    if epsilon > epsilon_final and i_episode%2!=0:
            epsilon *= epsilon_decay


    
    # env1 and entrance stimulus = 'A', Q1


    if i_episode%2==0 and entrance_stimulus == 'A' :
      state = env1.reset()
      step = 0
      reward = 0

      while True:
        step += 1

        random_for_epsilon = np.random.randn()

        action = choose_action(Q1, state, epsilon)
        

         
        ## env gives reward and next state and whether we've reached terminal state upon taking action at current state.. 
        new_state, _ , done, info = env1.step(action)

        ##if you want reward penalized at for each timestep
        reward = rewarder1(new_state, reward, entrance_stimulus)

        
        # filling the Q Table - 
        
        Q1.loc[state, action] = (1- learning_rate)*(Q1.at[state, action]) + (learning_rate)*(reward+ gamma*(np.max(Q1.loc[new_state, [0,2]])))
        
        # Setting new state for next action
        state = new_state
        env1.render()

        if done:
          print('Episode: {} Reward: {} Steps Taken: {} , Epsilon: {}, Entrance_Stimulus: {}, Environment: {}'.format(i_episode,reward, step,  epsilon, entrance_stimulus, map1  ))
          steps_total.append(step)
          break


      # env1 and entrance stimulus = 'B' , Q2


    elif i_episode%2==0 and entrance_stimulus == 'B':
      state = env1.reset()
      step = 0
      reward = 0

      while True:
        step += 1

        random_for_epsilon = np.random.randn()

        action = choose_action(Q2, state, epsilon)
        

         
        ## env gives reward and next state and whether we've reached terminal state upon taking action at current state.. 
        new_state, _ , done, info = env1.step(action)

        ##if you want reward penalized at for each timestep
        reward = rewarder1(new_state, reward, entrance_stimulus)

        
        # filling the Q Table - 
        
        Q2.loc[state, action] = (1- learning_rate)*(Q2.at[state, action]) + (learning_rate)*(reward+ gamma*(np.max(Q2.loc[new_state, [0,2]])))
        
        # Setting new state for next action
        state = new_state
        env2.render()


        if done:
          print('Episode: {} Reward: {} Steps Taken: {} , Epsilon: {}, Entrance_Stimulus: {}, Environment:{} '.format(i_episode,reward, step,  epsilon, entrance_stimulus, map2 ))
          steps_total.append(step)
          break
    

    # env2 and entrance stimulus = 'A' , Q3

    elif i_episode%2!=0 and entrance_stimulus == 'A':
      state = env2.reset()
      step = 0
      reward = 0

      while True:
        step += 1

        random_for_epsilon = np.random.randn()

        action = choose_action(Q3, state, epsilon)
        

         
        ## env gives reward and next state and whether we've reached terminal state upon taking action at current state.. 
        new_state, _ , done, info = env2.step(action)

        ##if you want reward penalized at for each timestep
        reward = rewarder2(new_state, reward, entrance_stimulus)

        
        # filling the Q Table - 
        
        Q3.loc[state, action] = (1- learning_rate)*(Q3.at[state, action]) + (learning_rate)*(reward+ gamma*(np.max(Q3.loc[new_state, [0,2]])))
        
        # Setting new state for next action
        state = new_state
        env2.render()


        if done:
          print('Episode: {} Reward: {} Steps Taken: {} , Epsilon: {}, Entrance_Stimulus: {}, Environment:{} '.format(i_episode,reward, step,  epsilon, entrance_stimulus, map2 ))
          steps_total.append(step)
          break
    

    #  env2 and entrance stimulus = 'B', Q4
    elif i_episode%2!=0 and entrance_stimulus == 'B':
      state = env2.reset()
      step = 0
      reward = 0

      while True:
        step += 1

        random_for_epsilon = np.random.randn()

        action = choose_action(Q4, state, epsilon)
        

         
        ## env gives reward and next state and whether we've reached terminal state upon taking action at current state.. 
        new_state, _ , done, info = env2.step(action)

        ##if you want reward penalized at for each timestep
        reward = rewarder2(new_state, reward, entrance_stimulus)

        
        # filling the Q Table - 
        
        Q4.loc[state, action] = (1- learning_rate)*(Q4.at[state, action]) + (learning_rate)*(reward+ gamma*(np.max(Q4.loc[new_state, [0,2]])))
        
        # Setting new state for next action
        state = new_state
        env2.render()


        if done:
          print('Episode: {} Reward: {} Steps Taken: {} , Epsilon: {}, Entrance_Stimulus: {}, Environment:{} '.format(i_episode,reward, step,  epsilon, entrance_stimulus, map2 ))
          steps_total.append(step)
          break


Streaming output truncated to the last 5000 lines.
GBSAG
Episode: 573 Reward: 90 Steps Taken: 2 , Epsilon: 0.09990224980716131, Entrance_Stimulus: B, Environment:['GBSAG'] 
  (Left)
GBSAG
  (Left)
GBSAG
Episode: 574 Reward: 90 Steps Taken: 2 , Epsilon: 0.09990224980716131, Entrance_Stimulus: B, Environment:['GBSAG'] 
  (Left)
GBSAG
  (Left)
GBSAG
Episode: 575 Reward: 90 Steps Taken: 2 , Epsilon: 0.09990224980716131, Entrance_Stimulus: B, Environment:['GBSAG'] 
  (Left)
GBSAG
  (Left)
GBSAG
Episode: 576 Reward: 90 Steps Taken: 2 , Epsilon: 0.09990224980716131, Entrance_Stimulus: B, Environment:['GBSAG'] 
  (Left)
GBSAG
  (Left)
GBSAG
Episode: 577 Reward: 90 Steps Taken: 2 , Epsilon: 0.09990224980716131, Entrance_Stimulus: B, Environment:['GBSAG'] 
  (Left)
GASBG
  (Left)
GASBG
Episode: 578 Reward: 90 Steps Taken: 2 , Epsilon: 0.09990224980716131, Entrance_Stimulus: A, Environment: ['GASBG']
  (Left)
GBSAG
  (Left)
GBSAG
Episode: 579 Reward: 90 Steps Taken: 2 , Epsilon: 0.099902249807161

In [32]:
# env1 and entrance stimulus = 'A'

# GASBG

Q1.loc[:, [0,2]]

,0,2
0,0.000864,0.000051
1,90.000778,43.900261
2,71.000700,-47.475576
3,-41.250552,-21.979529
4,0.000524,0.000136


In [33]:
# env1 and entrance stimulus = 'B'
# TBSAT

Q2.loc[:, [0,2]]

,0,2
0,0.000570,0.000064
1,-21.824687,43.886976
2,29.186117,52.808778
3,43.494017,71.982344
4,0.000403,0.000359


In [34]:
# env2 and entrance stimulus = 'A'

Q3.loc[:, [0,2]]

,0,2
0,0.000949,0.000336
1,-21.802744,16.044674
2,27.277795,67.389086
3,30.631831,89.800694
4,0.000549,0.000542


In [35]:
#env2 and entrance stimulus = 'B'

Q4.loc[:, [0,2]]

,0,2
0,0.000426,0.000220
1,89.982383,43.900125
2,70.544945,27.018484
3,43.753142,-20.005484
4,0.000108,0.000306


When the bee was trained, it was aware of the stimulus at the entrance,
and the locations of the two stimuli in the distinct chamber. 

Hypothesis: Through training it was exposed to four different environments, 

the algo above shows that if we set up four different Q-tables for each of those environments, a trained agent will be able to choose the correct 
Q table for performing the right action in each scenario. 
